In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 7
%matplotlib inline
import gc

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")
from operator import itemgetter

# Load bases

In [2]:
train_df = pd.read_csv('C:/Users/38063/Desktop/Loan_prediction/train.csv')
test_df = pd.read_csv('C:/Users/38063/Desktop/Loan_prediction/test.csv')
[print(df.shape) for df in [train_df,test_df]]

(7500, 18)
(2500, 17)


[None, None]

In [3]:
train_df.head()

,Id,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [4]:
test_df.head()

,Id,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
0,7500,Rent,NaN,4 years,0.0,9.0,12.5,220968.0,0.0,70.0,0.0,debt consolidation,Short Term,162470.0,105906.0,6813.0,NaN
1,7501,Rent,231838.0,1 year,0.0,6.0,32.7,55946.0,0.0,8.0,0.0,educational expenses,Short Term,78298.0,46037.0,2318.0,699.0
2,7502,Home Mortgage,1152540.0,3 years,0.0,10.0,13.7,204600.0,0.0,NaN,0.0,debt consolidation,Short Term,200178.0,146490.0,18729.0,7260.0
3,7503,Home Mortgage,1220313.0,10+ years,0.0,16.0,17.0,456302.0,0.0,70.0,0.0,debt consolidation,Short Term,217382.0,213199.0,27559.0,739.0
4,7504,Home Mortgage,2340952.0,6 years,0.0,11.0,23.6,1207272.0,0.0,NaN,0.0,debt consolidation,Long Term,777634.0,425391.0,42605.0,706.0


In [5]:
train_df.isnull().sum()

Id                                 0
Home Ownership                     0
Annual Income                   1557
Years in current job             371
Tax Liens                          0
Number of Open Accounts            0
Years of Credit History            0
Maximum Open Credit                0
Number of Credit Problems          0
Months since last delinquent    4081
Bankruptcies                      14
Purpose                            0
Term                               0
Current Loan Amount                0
Current Credit Balance             0
Monthly Debt                       0
Credit Score                    1557
Credit Default                     0
dtype: int64

In [6]:
test_df.isnull().sum()

Id                                 0
Home Ownership                     0
Annual Income                    513
Years in current job              86
Tax Liens                          0
Number of Open Accounts            0
Years of Credit History            0
Maximum Open Credit                0
Number of Credit Problems          0
Months since last delinquent    1358
Bankruptcies                       3
Purpose                            0
Term                               0
Current Loan Amount                0
Current Credit Balance             0
Monthly Debt                       0
Credit Score                     513
dtype: int64

Home Ownership - Домовладение\
Annual Income - Годовой доход\
Years in current job - Кол-во лет на текущей работе\
Tax Liens - Налоговые обязательства\
Number of Open Accounts - Количество открытых счетов\
Years of Credit History - Кол-во лет кредитная история\
Maximum Open Credit - Макс кол-во открытых кредитов\
Number of Credit Problems - Кол-во проблемных кредитов\
Months since last delinquent - Кол-вомесяцев с момента последнего просроченного платежа\
Bankruptcies - банкротство\
Purpose - Цель?\
Term - Срок\
Current Loan Amount - Текущая сумма кредита\
Current Credit Balance - Текущий кредитовый баланс\
Monthly Debt - Ежемесячный долг\
Credit Score - Кредитный рейтинг\
Credit Default - Кредитный дефолт

In [7]:
# 'Annual Income',
# 'Credit Score', - первый и второй - по одни и тем же клиентам пустые значения. Возможно первый входит как предиктор во второй

# Preprocesing

## fill NAN

In [8]:
for df in [train_df, test_df]:
    for f in ['Annual Income', 
              'Months since last delinquent',
              'Bankruptcies',
              ]:
        nf_ = f + '_flNull'
        df[nf_] = np.where(df[f].isnull(), 1, 0)
        print(nf_, df[nf_].isnull().sum())

Annual Income_flNull 0
Months since last delinquent_flNull 0
Bankruptcies_flNull 0
Annual Income_flNull 0
Months since last delinquent_flNull 0
Bankruptcies_flNull 0


In [9]:
# Annual Income
for df in [train_df,test_df]:
    for f in ['Annual Income']:
        median_ = df['Annual Income'].median()
        df[f]= df[f].fillna(median_)

In [10]:
for df in [train_df,test_df]:
    for f in ['Years in current job']:
        nf_ = f#+'_NAN_0'
        df[nf_] = df[f].fillna('0 years')
#         print(df[nf_].isnull().sum())

for df in [train_df,test_df]:
    for f in ['Months since last delinquent']:
        nf_ = f#+'_NAN_0'
        df[nf_] = df[f].fillna(0.0)
#         print(df[nf_].isnull().sum())

In [11]:
for df in [train_df,test_df]:
    for f in ['Bankruptcies']:
        nf_ = f#+'_NAN_0'
        df[nf_] = df[f].fillna(0.0)
#         print(df[nf_].isnull().sum())

In [12]:
gc.collect()

80

# Feature Engineering

In [13]:
for df in [train_df,test_df]:
    df['Term'] = np.where(df['Term'] == 'Long Term', 1, 0)
    df['b_75 Credit Score'] = np.where(df['Credit Score']>743, 1, 0)
    df['b_75 Current Loan Amount'] = np.where(df['Current Loan Amount']>5.198820e+05, 1, 0)

In [17]:
predictors = list(set(train_df.columns.tolist()) - set(['Id','Credit Default']))
target = 'Credit Default'
print(len(predictors))

21


In [18]:
obj = train_df[predictors].select_dtypes(include = 'O').columns.tolist()
num = list(set(predictors)-set(obj))
print ('obj = ', len(obj))
print ('num = ', len(num))

obj =  3
num =  18


In [19]:
bin_pred =[f for f in list(set(predictors)) if train_df[f].nunique(dropna=False)<=2]
len(bin_pred)

6

In [20]:
num = list(set(num)-set(bin_pred))
obj = list(set(obj)-set(bin_pred))
print ('num = ', len(num))
print ('obj = ', len(obj))

num =  12
obj =  3


In [21]:
bin_pred

['Bankruptcies_flNull',
 'Term',
 'Months since last delinquent_flNull',
 'Annual Income_flNull',
 'b_75 Credit Score',
 'b_75 Current Loan Amount']

In [22]:
obj

['Years in current job', 'Purpose', 'Home Ownership']

In [23]:
num

['Months since last delinquent',
 'Years of Credit History',
 'Credit Score',
 'Tax Liens',
 'Number of Open Accounts',
 'Annual Income',
 'Maximum Open Credit',
 'Monthly Debt',
 'Number of Credit Problems',
 'Current Loan Amount',
 'Bankruptcies',
 'Current Credit Balance']

In [24]:
num_pred = num.copy()
obj_pred = obj.copy()

In [25]:
for i in predictors:
    if i not in bin_pred+num_pred+obj_pred:
        print (i)

# LogisticRegression

In [26]:
X_train, X_test, y_train, y_test = train_test_split(train_df, train_df[target], test_size=0.2, random_state=42)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import  Pipeline
from sklearn.model_selection import cross_validate

class PassthroughTransformer:
    def __init__(self):
        pass
    def fit(self,X,y=None):
        if isinstance(X,pd.DataFrame):
            self.colnames = X.columns.tolist()
        else:
            self.colnames = [str(col) for col in np.arange(0,X.shape[0])]
        return self
    def transform(self,X):
        return X
    def get_feature_names(self):
        return self.colnames

In [28]:
obj_lg_pipe_woe = Pipeline([('woe', ce.WOEEncoder(handle_unknown='value'))])
tr_standart_zero=Pipeline([('imp', SimpleImputer(fill_value=0, strategy='constant'))])
tr_passthrough= Pipeline([('transf',PassthroughTransformer())])  
    
#все в 1 трансформе
transformers_f = [('obj_lg_pipe_woe',         obj_lg_pipe_woe,           obj_pred),  
                  ('tr_standart_zero',        tr_standart_zero,          num_pred), 
                  ('tr_passthrough',          tr_passthrough,            bin_pred)
                 ] 

al_pipe_feats = obj_pred+num_pred+bin_pred
                
print (len(al_pipe_feats), len(set(al_pipe_feats)))
transformer_com = ColumnTransformer (transformers=transformers_f)
COM_pipe = Pipeline([('transf', transformer_com),
                     ('scaler',MinMaxScaler()),
                     ('lg', LogisticRegression(n_jobs=-1))])

21 21


In [29]:
tscv = TimeSeriesSplit(n_splits=3) 

In [30]:
cv_scres = cross_validate(COM_pipe,
                          X_train[al_pipe_feats].reset_index(drop=True),
                          y_train.reset_index(drop=True),
                          return_train_score=True,
                          verbose=3,
                          cv=tscv,
                          n_jobs=3,
                          scoring = 'f1')
print (cv_scres['train_score'])
print (cv_scres['test_score'])
COM_pipe.fit(X_train[al_pipe_feats].reset_index(drop=True),y_train.reset_index(drop=True))
print ('Train', f1_score(y_train,COM_pipe.predict(X_train[al_pipe_feats])))
print ('Test', f1_score(y_test,COM_pipe.predict(X_test[al_pipe_feats])))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


[0.4171123  0.38432836 0.38428484]
[0.36363636 0.37546468 0.38345865]
Train 0.39282428702851885
Test 0.3767605633802817


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    3.4s finished


In [34]:
obj_lg_pipe_woe = Pipeline([('woe', ce.WOEEncoder(handle_unknown='value'))])
tr_standart_zero=Pipeline([('imp', SimpleImputer(fill_value=0, strategy='constant'))])
tr_passthrough= Pipeline([('transf',PassthroughTransformer())])  
    
#все в 1 трансформе
transformers_f = [('obj_lg_pipe_woe',         obj_lg_pipe_woe,           obj_pred),  
                  ('tr_standart_zero',        tr_standart_zero,          num_pred), 
                  ('tr_passthrough',          tr_passthrough,            bin_pred)
                 ] 

al_pipe_feats = obj_pred+num_pred+bin_pred
                
print (len(al_pipe_feats), len(set(al_pipe_feats)))
transformer_com = ColumnTransformer (transformers=transformers_f)
COM_pipe = Pipeline([('transf', transformer_com),
                     ('scaler',MinMaxScaler()),
                     ('lg', LogisticRegression(C = 0.03, class_weight = 'balanced', solver= 'lbfgs'))])

21 21


In [35]:
cv_scres = cross_validate(COM_pipe,
                          X_train[al_pipe_feats].reset_index(drop=True),
                          y_train.reset_index(drop=True),
                          return_train_score=True,
                          verbose=3,
                          cv=tscv,
                          n_jobs=3,
                          scoring = 'f1')
print (cv_scres['train_score'])
print (cv_scres['test_score'])
COM_pipe.fit(X_train[al_pipe_feats].reset_index(drop=True),y_train.reset_index(drop=True))
print ('Train', f1_score(y_train,COM_pipe.predict(X_train[al_pipe_feats])))
print ('Test', f1_score(y_test,COM_pipe.predict(X_test[al_pipe_feats])))
# [0.52497551 0.52315522 0.51890034]
# [0.50048685 0.52371134 0.54127482]
# Train 0.5270999229386077
# Test 0.5617103984450924

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


[0.52497551 0.52315522 0.51890034]
[0.50048685 0.52371134 0.54127482]
Train 0.5270999229386077
Test 0.5617103984450924


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.1s finished


In [36]:
coef_l1 = dict(zip(COM_pipe['lg'].coef_[0], al_pipe_feats))
coef_l1 = dict(sorted(coef_l1.items(), key=itemgetter(0), reverse=True))

d = {'REG_Coeficient': list(coef_l1.keys()), 'Feature': list(coef_l1.values())}
coef_df = pd.DataFrame(data=d)
print (coef_df.shape)
coef_df['abs'] = coef_df.REG_Coeficient.abs()
coef_df.sort_values('abs', ascending=False)


(21, 2)


,REG_Coeficient,Feature,abs
0,2.110413,Credit Score,2.110413
20,-1.702819,Current Loan Amount,1.702819
1,0.810541,Term,0.810541
2,0.556065,Annual Income_flNull,0.556065
19,-0.468685,Annual Income,0.468685
3,0.467363,Years in current job,0.467363
4,0.304868,Home Ownership,0.304868
5,0.261555,Purpose,0.261555
6,0.186265,Number of Open Accounts,0.186265
7,0.157035,Monthly Debt,0.157035


In [37]:
f_l1 = coef_df[abs(coef_df['REG_Coeficient']) >= 0.0000]['Feature'].tolist()
print(len(al_pipe_feats), len(f_l1))

21 21


In [40]:
# al_pipe_feats = list(set(al_pipe_feats)&set(f_l1))

In [38]:
COM_pipe.fit(train_df[al_pipe_feats].reset_index(drop=True),train_df[target].reset_index(drop=True))
print ('Train', f1_score(y_train,COM_pipe.predict(X_train[al_pipe_feats])))
print ('Test', f1_score(y_test,COM_pipe.predict(X_test[al_pipe_feats])))
print ('All', f1_score(train_df[target],COM_pipe.predict(train_df[al_pipe_feats])))

Train 0.5276420673695037
Test 0.5596868884540117
All 0.5343107310120138


In [39]:
test_df[target] = COM_pipe.predict(test_df[al_pipe_feats])
test_df[['Id','Credit Default']].to_csv('Submission_15.csv',index=False)